In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from db_utils import query_analytics_store, query_hive_ssh,get_hive_timespan

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Group Requests By (IP, UA, XFF) in Hive

In [55]:
query = """
DROP TABLE IF EXISTS ellery.survey_qa_%(ext)s;
CREATE TABLE ellery.survey_qa_%(ext)s
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t'
STORED AS TEXTFILE

AS SELECT
    client_ip,
    user_agent,
    x_forwarded_for,
    CONCAT_WS('||', COLLECT_LIST(request)) as requests,
    day_ts
FROM
    (SELECT
        client_ip,
        user_agent,
        x_forwarded_for,
        CONCAT(ts, '|', referer, '|', uri_path ) request,
        CONCAT(year,'-',LPAD(month,2,'0'),'-',LPAD(day,2,'0')) as day_ts
    FROM
        wmf.webrequest
    WHERE 
        is_pageview
        AND agent_type = 'user'
        AND %(time_conditions)s
        AND uri_host in ('en.wikipedia.org', 'en.m.wikipedia.org')
    ) a
GROUP BY
    client_ip,
    user_agent,
    x_forwarded_for,
    day_ts

"""

# Create One Hour Test Table

In [ ]:
start = '2016-02-17 01:00:00'
stop = '2016-02-17 01:59:00'
params = {'time_conditions': get_hive_timespan(start, stop, hour = True),
        'ext': 'hour'}
query_hive_ssh(query % params , 'qa', priority = True)

# Create One Day Test Table

In [57]:
start = '2016-02-17'
stop = '2016-02-17'
params = {'time_conditions': get_hive_timespan(start, stop, hour = False),
        'ext': 'day'}
query_hive_ssh(query % params , 'qa', priority = True)


DROP TABLE IF EXISTS ellery.survey_qa_day;
CREATE TABLE ellery.survey_qa
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '	'
STORED AS TEXTFILE

AS SELECT
    client_ip,
    user_agent,
    x_forwarded_for,
    CONCAT_WS('||', COLLECT_LIST(request)) as requests,
    day_ts
FROM
    (SELECT
        client_ip,
        user_agent,
        x_forwarded_for,
        CONCAT(ts, '|', referer, '|', uri_path ) request,
        CONCAT(year,'-',LPAD(month,2,'0'),'-',LPAD(day,2,'0')) as day_ts
    FROM
        wmf.webrequest
    WHERE 
        is_pageview
        AND agent_type = 'user'
        AND ((year=2016 AND month=2 AND day=17))
        AND uri_host in ('en.wikipedia.org', 'en.m.wikipedia.org')
    ) a
GROUP BY
    client_ip,
    user_agent,
    x_forwarded_for,
    day_ts




,client_ip,user_agent,x_forwarded_for,requests,day_ts
